In [ ]:
import cv2
import os
import numpy as np
from random import shuffle, sample

def update_frame(video, frame_idx, video_name, saved_frames):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = video.read()
    if ret:
        display_frame = frame.copy()
        cv2.putText(display_frame, f"Frame: {frame_idx} | Video: {video_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.7, (255, 255, 255), 2)
        
        if frame_idx in saved_frames:
            cv2.putText(display_frame, "SAVED", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.imshow("Video Frame", display_frame)
        
    return ret, frame

def main():
    folder_path = r"C:\Users\PC\Desktop\Paris\Videos"

    videos = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.avi')]

    if not videos:
        print("No videos found in the selected directory.")
        return

    certain = []
    possible = []
    
    broken = False

    for video_path in videos:
        saved_frames = set()
        
        video_name = os.path.basename(video_path)
        video = cv2.VideoCapture(video_path)
        frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_idx = 0

        while True:
            ret, frame = update_frame(video, frame_idx, video_name, saved_frames)
            if not ret:
                break

            key = cv2.waitKey(0) & 0xFF

            if key == ord('q'):  # Backward 100 frames
                frame_idx = max(0, frame_idx - 100)
            elif key == ord('e'):  # Forward 100 frames
                frame_idx = min(frame_count - 1, frame_idx + 100)
            elif key == ord('a'):  # Backward 1 frame
                frame_idx = max(0, frame_idx - 1)
            elif key == ord('d'):  # Forward 1 frame
                frame_idx = min(frame_count - 1, frame_idx + 1)
            elif key == ord('s'):  # Save current frame to 'certain'
                if ret:
                    certain.append(frame)
                    saved_frames.add(frame_idx)
            elif key == ord('k'):  # Save 21 frames to 'possible'
                for i in range(frame_idx - 10, frame_idx + 11):
                    if 0 <= i < frame_count:
                        video.set(cv2.CAP_PROP_POS_FRAMES, i)
                        ret, frame = video.read()
                        if ret:
                            possible.append(frame)
                            saved_frames.add(frame_idx)
            elif key == 27:  # ESC key
                video.release()
                cv2.destroyAllWindows()
                broken = True
                break

            # Automatically go to the next video if we reach the end of the current one
            if frame_idx == frame_count - 1:
                break

        video.release()
        
        if broken:
            break

    # Append 10% of "possible" frames to "certain" frames
    n = int(len(possible) * 0.1)
    sample_frames = sample(possible, n)
    certain.extend(sample_frames)

    # Shuffle the "certain" array
    shuffle(certain)
    
    print(len(certain))

    # Save as "to_train.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    frame_height, frame_width = certain[0].shape[:2]
    out = cv2.VideoWriter('to_train1.avi', fourcc, 30, (frame_width, frame_height))

    for frame in certain:
        out.write(frame)

    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
